<a href="https://colab.research.google.com/github/tyslas/CS5265-tyslas-nfl-spread-line-outcomes/blob/main/NFLSpreadAndLineOutcomes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Project to predict future NFL Spread & Line outcomes

## Author: Tito Yslas

### Background
I enjoy watching the NFL and playing fantasy football. I also like to place bets on games using apps like FanDuel. The purpose of this project is to increase my understanding of the NFL betting market and possibly create a machine learning model to give myself an edge next season.

### Project Description
I found a dataset from Kaggle titled [NFL scores and betting data](https://www.kaggle.com/datasets/tobycrabtree/nfl-scores-and-betting-data?resource=download). This dataset has over 13,000 samples and 17 features. The goal is to use this dataset to train a model that will predict the winner of a given game.

### Performance Metric
For my performance metric, I will aim for the model to have at least 70% accuracy.

## Import Libraries

In [5]:
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt

## Load Data

### Data Dictionary
- schedule_date: Date that the game took place. This column is a date in the format MM/DD/YYYY
- schedule_season: Year of the season began. NFL seasons start in the fall and end before the spring of the following year. So the 2023 season will refer to the years 2023-2024. This column is a number in the format YYYY
- schedule_week: Week of the NFL season. This column is either a number during regular season weeks or a string in playoff weeks. For the purposes of this project the playoff weeks will be converted to numbers and the schedule_playoff column will be used to determine whether the week is a playoff game or not
- schedule_playoff: This column is a boolean. FALSE is regular season and TRUE is playoffs
- team_home: Name of the home team. This column is a string
- score_home: Points scored by the home team. This column is a number
- score_away: Points scored by the away team. This column is a number
- winner: This column will be a feature derived from score_home and score_away columns to that will use one hot encoding - if team_home scores more points this will be a 1 - if team_home scores fewer points it will be a 0
- team_away: Name of the away team. This column is a string
- team_favorite_id: Acronym of the team that was determined most likely to win by the betting market. It is either two or three letters. For the purposes of this project this column will be changed to be either the team_home or team_away name
- team_home_favorite: this will represent the encoded team_favorite_id - if team_home is favored this column will be marked as a 1 - if it's a zero then we know that team_away is favored
- spread_favorite: The number of points that the favored team needs to win by for a bet placed on the spread of the favorite to win. This column will either be a negative number or zero
- over_under_line: The number of points that both teams combined need to score for a bet placed on the 'line' to win. This column is a positive number
- stadium: Name of the venue that the game is played
- stadium_neutral: This column is a boolean. FALSE is not a neutral venue and TRUE is a neutral venue - for the purposes of this project this column will be one hot encoded with a neutral venue being marked as a 1 and non-neutral marked as a 0
- weather_temperature: The temperature in Fahrenheit at the venue where the game is played. This column is a number
- weather_wind_mph: The speed of wind in miles per hour. This column is a number
- weather_humidity: The measurement of water vapor in the air during the game measured as a percentage. This column is a number
- weather_detail: Other information about the weather conditions - if the venue is indoor or the venue has a retractable roof. This column is a string


## Exploratory Data Analysis
### Questions to answer with EDA:
1. Which columns, if any, should I modify the data type to better train my model?
1. Which columns, if any, should I remove from the training and test data so that the model can be effectively trained?
1. Which columns, if any, should I remove or insert derived data for in the case that there is a lot of missing data?
1. What features could it make sense to introduce to improve the training and performance of my model?

In [6]:
from google.colab import drive
drive.mount('/content/drive/')
# change directory to relevant files
%cd /content/drive/MyDrive/vandy/nfl
# print all file names in directory
for file in os.listdir():
  print(file)

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/MyDrive/vandy/nfl
data_dictionary.csv
spreadspoke_scores.csv


In [7]:
scores = pd.read_csv('spreadspoke_scores.csv')

In [8]:
scores.info()
# scores.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13516 entries, 0 to 13515
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   schedule_date        13516 non-null  object 
 1   schedule_season      13516 non-null  int64  
 2   schedule_week        13516 non-null  object 
 3   schedule_playoff     13516 non-null  bool   
 4   team_home            13516 non-null  object 
 5   score_home           13516 non-null  int64  
 6   score_away           13516 non-null  int64  
 7   team_away            13516 non-null  object 
 8   team_favorite_id     11037 non-null  object 
 9   spread_favorite      11037 non-null  float64
 10  over_under_line      11027 non-null  object 
 11  stadium              13516 non-null  object 
 12  stadium_neutral      13516 non-null  bool   
 13  weather_temperature  12309 non-null  float64
 14  weather_wind_mph     12293 non-null  float64
 15  weather_humidity     8468 non-null  

### Answer to Question 1
- I think that it makes sense to modify both team_home and team_away to have the same IDs as team_favorite_id to make it easier for the model to indentify when team_home is the same or different as team_favorite_id
- Currently Pandas is indentifying schedule_date as an object, it could make sense to see if there's a way for Pandas to indentify this as a date
- Currently Pandas is indentifying schedule_week as an object. This is because some of the data in this column is in string format. It could make sense to modify the data of this column that is in a string to only be in int64 format and use the schedule_playoff column to be the soe determination of whether or not the schedule_week is a playoff game
- Currently Pandas is indentifying the over_under_line column as an object data type despite the fact that it should be a float. I will explore how to ensure that this column's data type is correctly identified
- The stadium_neutral column is currently a boolean type, I think I will convert this to use one hot encoding instead

### Answer to Question 2
- Based on my initial examination, I'm not sure if it makes sense to remove any of my columns from the data set on which I will train my model

In [9]:
scores.isna().sum() # number of missing values for each column

schedule_date              0
schedule_season            0
schedule_week              0
schedule_playoff           0
team_home                  0
score_home                 0
score_away                 0
team_away                  0
team_favorite_id        2479
spread_favorite         2479
over_under_line         2489
stadium                    0
stadium_neutral            0
weather_temperature     1207
weather_wind_mph        1223
weather_humidity        5048
weather_detail         10597
dtype: int64

### Answer to Question 3
- the columns with missing data include team_favorite_id, spread_favorite, over_under_line, weather_temperature, weather_wind_mph, weather_humidity, and weather_detail. these columns have many missing values because this data was not collected in earlier seasons. for example, there is minimal team_favorite_id information collected from the 1978 schedule_season and previous to that likely because of the lack of public betting information before that time
- for the missing data I don't think that it makes sense to remove these columns, however it might make sense to only train and test on the observations from the 1979 schedule_season and beyond

### Answer to Question 4
- I think it makes sense to introduce/derive three different target columns for understanding the performance of the model
- The three target columns I am thinking about introducing are derived from score_home, score_away, and over_under_line
- These targets would be one hot encoded as team_home_win, team_home_cover_spread, and cover_line
- team_home_win would be a 1 if team_home wins or a 0 if they lose
- team_home_cover_spread would be a 1 if they cover the spread_favorite and a 0 if they don't
- cover_line would be a 1 if score_home + score_way is greater than the over_under_line and a 0 if it's less than

## Feature Engineering
1. convert `schedule_date` column to actually be read in as a date/time object instead of generic object
1. convert `schedule_week` to only be numbers - this could be more challenging than I initially thought because there will not be direct mappings for the outliers of `Division`, `Wild Card`, `Conference`, and `Superbowl` because the NFL has expanded the numbers of games played during the regular season over the years and added in the `Wildcard` games
1. convert the `schedule_playoff` column from true/false to 1/0
1. add a target/feature of `winning_team` to be derived from `score_home` and `score_away` to make it easier to determine how the model performs
1. convert all `team_home` and `team_away` entries to the acronym identifiers
1. drop rows 0 - 2499 because they don't have data for `team_favorite_id`, `spread_favorite` and `over_under_line`
1. create new target column of `favorite_won` with 1 for true and 0 for false
1. create new target column of `spread_favorite_covered` with 1 for true and 0 for false
1. create new target column of `over_under_covered`
1. convert the `weather_detail` column to a categorical variable that is one-hot encoded

In [10]:
# convert schedule_date to proper date type
scores['schedule_date'] = pd.to_datetime(scores['schedule_date'])
schedule_date_data_type = scores['schedule_date'].dtype
print('data type of schedule_date: ', schedule_date_data_type)

data type of schedule_date:  datetime64[ns]


In [ ]:
# convert schedule_week to only be numbers
scores['schedule_week'].value_counts()
# this may require a fair amount of time spent manually mapping week numbers

In [ ]:
# convert the schedule_playoff column from true/false into 1/0
scores['schedule_playoff'] = scores['schedule_playoff'].astype(int)
display(scores)

In [ ]:
# convert all team_home and team_away entries to the acronym identifiers

# map of team names and their corresponding IDs
team_ids = {
    'San Francisco 49ers': 'SF',
    'Dallas Cowboys': 'DAL',
    'Pittsburgh Steelers': 'PIT',
    'Green Bay Packers': 'GB',
    'Philadelphia Eagles': 'PHI',
    'Minnesota Vikings': 'MIN',
    'Denver Broncos': 'DEN',
    'Miami Dolphins': 'MIA',
    'Kansas City Chiefs': 'KC',
    'Buffalo Bills': 'BUF',
    'Chicago Bears': 'CHI',
    'New York Giants': 'NYG',
    'Atlanta Falcons': 'ATL',
    'New Orleans Saints': 'NO',
    'New York Jets': 'NYJ',
    'Detroit Lions': 'DET',
    'Cincinnati Bengals': 'CIN',
    'New England Patriots': 'NE',
    'Washington Redskins': 'WAS',
    'Cleveland Browns': 'CLE',
    # should be SD but all the data uses LAC
    'San Diego Chargers': 'LAC',
    'Seattle Seahawks': 'SEA',
    'Tampa Bay Buccaneers': 'TB',
    # should be OAK but all the data uses LVR
    'Oakland Raiders': 'LVR',
    'Indianapolis Colts': 'IND',
    'Los Angeles Rams': 'LAR',
    'Arizona Cardinals': 'ARI',
    # should be HOU but all the data uses TEN
    'Houston Oilers': 'TEN',
    'Carolina Panthers': 'CAR',
    'Jacksonville Jaguars': 'JAX',
    'Baltimore Ravens': 'BAL',
    'Tennessee Titans': 'TEN',
    # should be STL but all the data uses LAR
    'St. Louis Rams': 'LAR',
    'Houston Texans': 'HOU',
    # should be STL but all the data uses ARI
    'St. Louis Cardinals': 'ARI',
    'Baltimore Colts': 'BAL',
    # should be LAR but all the data uses LVR
    'Los Angeles Raiders': 'LVR',
    'Los Angeles Chargers': 'LAC',
    # should be PHX but all the data uses ARI
    'Phoenix Cardinals': 'ARI',
    # 'Boston Patriots': '', the franchise changed the name of their team to the New England Patriots in 1971
    # the data does not have any team_favorite_id listed before the 12/24/78 in the 1978 season (row 2494)
    'Las Vegas Raiders': 'LVR',
    'Washington Football Team': 'WAS',
    'Tennessee Oilers': 'TEN',
    'Washington Commanders': 'WAS',
}

scores['team_home'] = scores['team_home'].replace(team_ids)
scores['team_away'] = scores['team_away'].replace(team_ids)

scores.loc[2492:2502]

In [ ]:
# add a target/feature of 'winning_team'
# Derive a new column based on the comparison of two existing columns
def determine_winner(row):
  if row['score_home'] > row['score_away']:
    val = row['team_home']
  elif row['score_home'] == row['score_away']:
    val = 'tie'
  else:
    val = row['team_away']
  return val

scores['winning_team'] = scores.apply(determine_winner, axis=1)
print(scores['winning_team'].tail(20))
print('number of ties:', scores['winning_team'].value_counts()['tie'])

In [16]:
# drop rows 0 - 2499 because they don't have data for team_favorite_id, spread_favorite and over_under_line
start_index = 0
end_index = 2500

scores_dropped = scores.drop(scores.index[start_index:end_index])
scores_dropped.reset_index(drop = True, inplace = True)
scores_dropped.head(20)

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail,winning_team
0,1979-01-21,1978,Superbowl,1,DAL,31,35,PIT,PIT,-3.5,37,Orange Bowl,True,71.0,18.0,84.0,rain,PIT
1,1979-09-01,1979,1,0,TB,31,16,DET,TB,-3.0,30,Houlihan's Stadium,False,79.0,9.0,87.0,NaN,TB
2,1979-09-02,1979,1,0,BUF,7,9,MIA,MIA,-5.0,39,Ralph Wilson Stadium,False,74.0,15.0,74.0,NaN,MIA
3,1979-09-02,1979,1,0,CHI,6,3,GB,CHI,-3.0,31,Soldier Field,False,78.0,11.0,68.0,NaN,CHI
4,1979-09-02,1979,1,0,DEN,10,0,CIN,DEN,-3.0,31.5,Mile High Stadium,False,69.0,6.0,38.0,NaN,DEN
5,1979-09-02,1979,1,0,KC,14,0,BAL,KC,-1.0,37,Arrowhead Stadium,False,76.0,8.0,71.0,NaN,KC
6,1979-09-02,1979,1,0,LAR,17,24,LVR,LAR,-4.0,36.5,Anaheim Stadium,False,70.0,10.0,77.0,NaN,LVR
7,1979-09-02,1979,1,0,MIN,28,22,SF,MIN,-7.0,32,Metropolitan Stadium,False,70.0,11.0,67.0,NaN,MIN
8,1979-09-02,1979,1,0,NO,34,40,ATL,NO,-5.0,32,Louisiana Superdome,False,72.0,0.0,NaN,indoor,ATL
9,1979-09-02,1979,1,0,NYJ,22,25,CLE,NYJ,-2.0,41,Giants Stadium,False,73.0,10.0,76.0,NaN,CLE


In [20]:
# create new target column of favorite_won with 1 for true and 0 for false
# the favorite_won column is returning an float instead of an int -- need to fix
# scores_dropped.drop('favorite_won', inplace = True)

def determine_favorite_won(row):
  result = 1
  if row['winning_team'] != row['team_favorite_id']:
    result = 0
  return int(result)

scores_dropped['favorite_won'] = scores_dropped.apply(determine_favorite_won, axis = 1)
scores_dropped.loc['favorite_won'] = scores_dropped['favorite_won'].astype(int)
scores_dropped.head()

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail,winning_team,favorite_won
0,1979-01-21,1978.0,Superbowl,1.0,DAL,31.0,35.0,PIT,PIT,-3.5,37,Orange Bowl,1.0,71.0,18.0,84.0,rain,PIT,1.0
1,1979-09-01,1979.0,1,0.0,TB,31.0,16.0,DET,TB,-3.0,30,Houlihan's Stadium,0.0,79.0,9.0,87.0,NaN,TB,1.0
2,1979-09-02,1979.0,1,0.0,BUF,7.0,9.0,MIA,MIA,-5.0,39,Ralph Wilson Stadium,0.0,74.0,15.0,74.0,NaN,MIA,1.0
3,1979-09-02,1979.0,1,0.0,CHI,6.0,3.0,GB,CHI,-3.0,31,Soldier Field,0.0,78.0,11.0,68.0,NaN,CHI,1.0
4,1979-09-02,1979.0,1,0.0,DEN,10.0,0.0,CIN,DEN,-3.0,31.5,Mile High Stadium,0.0,69.0,6.0,38.0,NaN,DEN,1.0


In [33]:
# create new target column of spread_favorite_covered with 1 for true and 0 for false
# the favorite_won column is returning an float instead of an int -- need to fix

def determine_favorite_covered_spread(row):
  return 1 if row['score_home'] - row['score_away'] > abs(row['spread_favorite']) else 0

scores_dropped['spread_favorite_covered'] = scores_dropped.apply(determine_favorite_covered_spread, axis = 1)
display(scores_dropped.head(20))

counts = scores_dropped['spread_favorite_covered'].value_counts()
favorite_missed = counts.values[0]
favorite_covered = counts.values[1]
percent_favorite_won = round((favorite_covered / (favorite_covered + favorite_missed)) * 100, 2)

print(f"the favorite team covered the spread {percent_favorite_won}% of the time")

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail,winning_team,favorite_won,spread_favorite_covered
0,1979-01-21,1978.0,Superbowl,1.0,DAL,31.0,35.0,PIT,PIT,-3.5,37,Orange Bowl,1.0,71.0,18.0,84.0,rain,PIT,1.0,0
1,1979-09-01,1979.0,1,0.0,TB,31.0,16.0,DET,TB,-3.0,30,Houlihan's Stadium,0.0,79.0,9.0,87.0,NaN,TB,1.0,1
2,1979-09-02,1979.0,1,0.0,BUF,7.0,9.0,MIA,MIA,-5.0,39,Ralph Wilson Stadium,0.0,74.0,15.0,74.0,NaN,MIA,1.0,0
3,1979-09-02,1979.0,1,0.0,CHI,6.0,3.0,GB,CHI,-3.0,31,Soldier Field,0.0,78.0,11.0,68.0,NaN,CHI,1.0,0
4,1979-09-02,1979.0,1,0.0,DEN,10.0,0.0,CIN,DEN,-3.0,31.5,Mile High Stadium,0.0,69.0,6.0,38.0,NaN,DEN,1.0,1
5,1979-09-02,1979.0,1,0.0,KC,14.0,0.0,BAL,KC,-1.0,37,Arrowhead Stadium,0.0,76.0,8.0,71.0,NaN,KC,1.0,1
6,1979-09-02,1979.0,1,0.0,LAR,17.0,24.0,LVR,LAR,-4.0,36.5,Anaheim Stadium,0.0,70.0,10.0,77.0,NaN,LVR,0.0,0
7,1979-09-02,1979.0,1,0.0,MIN,28.0,22.0,SF,MIN,-7.0,32,Metropolitan Stadium,0.0,70.0,11.0,67.0,NaN,MIN,1.0,0
8,1979-09-02,1979.0,1,0.0,NO,34.0,40.0,ATL,NO,-5.0,32,Louisiana Superdome,0.0,72.0,0.0,NaN,indoor,ATL,0.0,0
9,1979-09-02,1979.0,1,0.0,NYJ,22.0,25.0,CLE,NYJ,-2.0,41,Giants Stadium,0.0,73.0,10.0,76.0,NaN,CLE,0.0,0


the favorite team covered the spread 40.52% of the time


In [37]:
# create new target column of over_under_covered

# need to cast over_under_line column to a float -- pandas recognizes it as a string
scores_dropped['over_under_line'] = pd.to_numeric(scores_dropped['over_under_line'], errors='coerce')
# scores_dropped['over_under_line'].astype(float)
print(scores_dropped['over_under_line'].dtypes)
def determine_over_under_covered(row):
  return 1 if row['score_home'] + row['score_away'] > row['over_under_line'] else 0

scores_dropped['over_under_covered'] = scores_dropped.apply(determine_over_under_covered, axis = 1)
display(scores_dropped[['score_home', 'score_away', 'over_under_line', 'over_under_covered']].head(20))

counts = scores_dropped['over_under_covered'].value_counts()
over_missed = counts.values[0]
over_covered = counts.values[1]
percent_over_covered = round((over_covered / (over_covered + over_missed)) * 100, 2)

print(f"the over hit {percent_over_covered}% of the time")

float64


,score_home,score_away,over_under_line,over_under_covered
0,31.0,35.0,37.0,1
1,31.0,16.0,30.0,1
2,7.0,9.0,39.0,0
3,6.0,3.0,31.0,0
4,10.0,0.0,31.5,0
5,14.0,0.0,37.0,0
6,17.0,24.0,36.5,1
7,28.0,22.0,32.0,1
8,34.0,40.0,32.0,1
9,22.0,25.0,41.0,1


the over hit 47.96% of the time


In [49]:
# convert the weather_detail column to categorical variable that is one-hot encoded

counts = scores_dropped['weather_detail'].value_counts()

totalWeatherDetails = 0
for count in counts:
  totalWeatherDetails += count
print(f"total number of rows with weather details: {totalWeatherDetails}")
numRows = scores_dropped.shape[0]
print(f"number of rows: {numRows}")

one_hot_encoded = pd.get_dummies(scores_dropped['weather_detail'], prefix = 'conditions')
scores_encoded = pd.concat([scores_dropped, one_hot_encoded], axis = 1)

scores_encoded.head(20)

total number of rows with weather details: 2758
number of rows: 11017


,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,...,spread_favorite_covered,over_under_covered,conditions_fog,conditions_indoor,conditions_rain,conditions_rain | fog,conditions_retractable (open roof),conditions_snow,conditions_snow | Freezing rain,conditions_snow | fog
0,1979-01-21,1978.0,Superbowl,1.0,DAL,31.0,35.0,PIT,PIT,-3.5,...,0,1,0,0,1,0,0,0,0,0
1,1979-09-01,1979.0,1,0.0,TB,31.0,16.0,DET,TB,-3.0,...,1,1,0,0,0,0,0,0,0,0
2,1979-09-02,1979.0,1,0.0,BUF,7.0,9.0,MIA,MIA,-5.0,...,0,0,0,0,0,0,0,0,0,0
3,1979-09-02,1979.0,1,0.0,CHI,6.0,3.0,GB,CHI,-3.0,...,0,0,0,0,0,0,0,0,0,0
4,1979-09-02,1979.0,1,0.0,DEN,10.0,0.0,CIN,DEN,-3.0,...,1,0,0,0,0,0,0,0,0,0
5,1979-09-02,1979.0,1,0.0,KC,14.0,0.0,BAL,KC,-1.0,...,1,0,0,0,0,0,0,0,0,0
6,1979-09-02,1979.0,1,0.0,LAR,17.0,24.0,LVR,LAR,-4.0,...,0,1,0,0,0,0,0,0,0,0
7,1979-09-02,1979.0,1,0.0,MIN,28.0,22.0,SF,MIN,-7.0,...,0,1,0,0,0,0,0,0,0,0
8,1979-09-02,1979.0,1,0.0,NO,34.0,40.0,ATL,NO,-5.0,...,0,1,0,1,0,0,0,0,0,0
9,1979-09-02,1979.0,1,0.0,NYJ,22.0,25.0,CLE,NYJ,-2.0,...,0,1,0,0,0,0,0,0,0,0


## Train/Test Split